In [1]:
# === CHECK MARTS (Pandas) ===============================================
# Use in Jupyter (cell by cell) or save as notebooks/check_marts.py and run.
# ------------------------------------------------------------------------

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# ---- Paths (edit if needed) --------------------------------------------
BASE_DIR = Path(__file__).resolve().parents[2] if "__file__" in globals() else Path("..").resolve()
DATA_DIR = BASE_DIR / "data"
MARTS_DIR = DATA_DIR / "marts"

print(MARTS_DIR)

# Partitioned Parquet directories (written by your Spark marts job)
FACT_BY_TYPE_DIR      = MARTS_DIR / "fact_prices"            # yearly agg by (district, property_type, year)
FACT_BY_DISTRICT_DIR  = MARTS_DIR / "fact_prices_district"   # yearly agg by (district, year)

# Single Parquet (coalesced to 1) with 5y bounds
BOUNDS_5Y_PATH        = MARTS_DIR / "agg_bounds_5y"

TRANSFORMED = DATA_DIR / "transformed"

RAW = DATA_DIR / "raw" / "pp_complete.csv"

T_CSV = BASE_DIR / "ppdata_with_header.csv"

# Show full floats without scientific notation
pd.set_option("display.float_format", "{:,.0f}".format)

/Users/renatopavlekovic/Documents/Digital Futures/property-pulse-uk/data/marts


In [19]:
# ---- Helpers ------------------------------------------------------------
def must_exist(path: Path, kind: str = "path"):
    if not path.exists():
        raise FileNotFoundError(f"Missing {kind}: {path}")
    return path

def summarize_df(name: str, df: pd.DataFrame, n: int = 5):
    print(f"\n=== {name}: shape={df.shape} ===")
    print(df.head(n))
    print("\n-- dtypes --")
    print(df.dtypes)
    print("\n-- describe (numeric) --")
    print(df.describe().T)
    # Basic null check
    nulls = df.isna().sum()
    if nulls.any():
        print("\n-- null counts --")
        print(nulls[nulls > 0].sort_values(ascending=False))

def coerce_numeric(df: pd.DataFrame, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

In [2]:
# ---- Load marts ---------------------------------------------------------
#must_exist(FACT_BY_TYPE_DIR, "parquet dir")
#must_exist(FACT_BY_DISTRICT_DIR, "parquet dir")
#must_exist(BOUNDS_5Y_PATH, "parquet dir")

df_type = pd.read_parquet(FACT_BY_TYPE_DIR)
df_dist = pd.read_parquet(FACT_BY_DISTRICT_DIR)
df_bounds = pd.read_parquet(BOUNDS_5Y_PATH)

df_parq = pd.read_parquet(TRANSFORMED)






In [5]:


# Read raw CSV (headerless)
#df_csv = pd.read_csv(RAW, header=None, names=columns)
#df_full_csv = pd.read_csv(T_CSV)

# Save with headers
#df_csv.to_csv("ppdata_with_header.csv", index=False)


# Convert transfer_datetime to datetime
df_full_csv["transfer_datetime"] = pd.to_datetime(df_full_csv["transfer_datetime"], errors="coerce")


In [21]:
# ---- Summaries ----------------------------------------------------------
summarize_df("FACT_BY_TYPE (district, property_type, year)", df_type)
summarize_df("FACT_BY_DISTRICT (district, year)", df_dist)
summarize_df("BOUNDS_5Y (min/max per district over last 5y)", df_bounds)


=== FACT_BY_TYPE (district, property_type, year): shape=(18173, 5) ===
       district property_type  avg_price  txn_count  year
0   EAST DORSET             D    113,952        150  1995
1    BIRMINGHAM             T     40,992        747  1995
2   TEST VALLEY             T     57,806         70  1995
3  PETERBOROUGH             D     68,676         83  1995
4      TRAFFORD             T     49,421        113  1995

-- dtypes --
district          object
property_type     object
avg_price        float64
txn_count          int64
year               int64
dtype: object

-- describe (numeric) --
           count    mean     std   min     25%     50%     75%        max
avg_price 18,173 237,662 324,247   380 106,333 169,894 274,995 12,930,000
txn_count 18,173      59      73     1      11      38      80        879
year      18,173   2,010       9 1,995   2,004   2,010   2,013      2,025

=== FACT_BY_DISTRICT (district, year): shape=(5578, 4) ===
                district  avg_price  txn_coun

In [ ]:
# Filter Colchester rows and in 2022 (there should be 4628 transactions)
check_ch = df_full_csv[(df_full_csv["town"].str.upper() == "COLCHESTER") & 
                   (df_full_csv["county"].str.upper() == "ESSEX") & 
                   (df_full_csv["transfer_datetime"].dt.year == 2022)]

check_ch.count()

transaction_id       4880
price                4880
transfer_datetime    4880
postcode             4867
property_type        4880
new_build            4880
tenure               4880
paon                 4880
saon                  472
street               4875
locality             2471
town                 4880
district             4880
county               4880
ppd_category         4880
record_status        4880
dtype: int64

In [24]:
# Filter Colchester rows and in 2022 (there should be 4628 transactions)
check_ch = df_parq[(df_parq["town"].str.upper() == "COLCHESTER") & 
                   (df_parq["county"].str.upper() == "ESSEX") & 
                   (df_parq["year"] == 2022)]

check_ch.count()

transaction_id       4628
price                4628
transfer_datetime    4628
postcode             4628
property_type        4628
new_build            4628
tenure               4628
paon                 4628
saon                  438
street               4625
locality             2326
town                 4628
district             4628
county               4628
ppd_category         4628
record_status        4628
transfer_ts          4628
transfer_date        4628
year                 4628
dtype: int64

In [8]:
df_full_csv.count()

transaction_id       30365754
price                30365754
transfer_datetime    30365754
postcode             30316542
property_type        30365754
new_build            30365754
tenure               30365754
paon                 30361570
saon                  3626679
street               29883553
locality             18987870
town                 30365754
district             30365754
county               30365754
ppd_category         30365754
record_status        30365754
dtype: int64

In [22]:
df_parq.count()

transaction_id       29789443
price                29789443
transfer_datetime    29789443
postcode             29773506
property_type        29789443
new_build            29789443
tenure               29789443
paon                 29785259
saon                  3526921
street               29354290
locality             18750529
town                 29789443
district             29789443
county               29789443
ppd_category         29789443
record_status        29789443
transfer_ts          29789443
transfer_date        29789443
year                 29789443
dtype: int64

In [4]:
# Filter Manchester rows and only before 2025
scot = df_parq[(df_parq["district"].str.upper() == "HIGHLAND")]


# Show the result
display(scot)

,transaction_id,price,transfer_datetime,postcode,property_type,new_build,tenure,paon,saon,street,locality,town,district,county,ppd_category,record_status,transfer_ts,transfer_date,year


In [38]:
df_full["district"].unique()

array(['WIGAN', 'SHEPWAY', 'WOLVERHAMPTON', 'CEREDIGION', 'HOUNSLOW',
       'PORTSMOUTH', 'NORTH WEST LEICESTERSHIRE', 'SOUTH NORFOLK',
       'HORSHAM', 'ENFIELD', 'CASTLE POINT', 'BROXBOURNE', 'CARDIFF',
       'CROYDON', 'ROCHESTER UPON MEDWAY', 'REIGATE AND BANSTEAD',
       'AYLESBURY VALE', 'EREWASH', 'HAMMERSMITH AND FULHAM', 'FENLAND',
       'DARLINGTON', 'SEDGEFIELD', 'CRAWLEY', 'ADUR', 'MANCHESTER',
       'THREE RIVERS', 'ARUN', 'GATESHEAD', 'BARNET', 'HARINGEY',
       'MID DEVON', 'BARNSLEY', 'COTSWOLD', 'ROSSENDALE', 'BRECKLAND',
       'LEEDS', 'WAKEFIELD', 'PORT TALBOT', 'RESTORMEL', 'SANDWELL',
       'MAIDSTONE', 'SOUTH STAFFORDSHIRE', 'ROTHERHAM', 'CARADON',
       'WREXHAM MAELOR', 'BLACKBURN', 'KINGSWOOD', 'SOUTH KESTEVEN',
       'GLOUCESTER', 'RICHMOND UPON THAMES', 'CHARNWOOD', 'KENNET',
       'CHELTENHAM', 'BEXLEY', 'BURY', 'KERRIER', 'TAMESIDE',
       'WINCHESTER', 'EAST HERTFORDSHIRE', 'NORWICH', 'TOWER HAMLETS',
       'LEICESTER', 'STOKE-ON-TRENT', 'TAF

In [ ]:
from config.streamlit_config import MART_FACT_BY_DISTRICT